### B-S方程定价

In [1]:
import math
from enum import Enum
class PayoffType(str, Enum):
    Call = 'Call'
    Put = 'Put'
def cnorm(x):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0
def bsPrice(S, r, vol, payoffType, K, T):
    fwd = S * math.exp(r * T)
    stdev = vol * math.sqrt(T)
    d1 = math.log(fwd / K) / stdev + stdev / 2
    d2 = d1 - stdev
    if payoffType == PayoffType.Call:
        return math.exp(-r * T) * (fwd * cnorm(d1) - cnorm(d2) * K)
    elif payoffType == PayoffType.Put:
        return math.exp(-r * T) * (K * cnorm(-d2) - cnorm(-d1) * fwd)
    else:
        raise Exception("not supported payoff type", payoffType)
    

In [2]:
# test ---
S, r, vol, K, T = 100, 0.01, 0.2, 105, 1.0
print("blackPrice: ", bsPrice(S, r, vol, PayoffType.Call, T, K))

blackPrice:  99.65683502132815


### 区间缩放算法

In [5]:
def rootBacketing(f, a, b, maxIter, factor):
    for k in range(maxIter):
        if f(a) * f(b) < 0:
            return (a, b)
        if abs(f(a)) < abs(f(b)):
            a += factor * (a - b) # if f(a) is closer to 0, change a
        else:
            b += factor * (b -a) # if f(b) is closer to 0, change b
    return (a, b)

In [6]:
import math
foo = lambda x: math.exp(x) - 5
a = 3.4
b = 5.78
(a_, b_) = rootBacketing(foo, a, b, 50, 1.6)
print(a_, b_)

-0.4080000000000008 5.78


In [10]:
foo(a_)*foo(b_)

-1381.8278230282199

### 寻根算法

#### 二分法

In [11]:
def bisect(f, a, b, tol):
    assert (a < b, f(a)*f(b) < 0)
    c = (a+b) / 2
    while (b-a)/2 > tol:
        print(f"({a}, {b})")
        c = (a+b)/2
        if abs(f(c)) < tol:
            return c
        else:
            if f(a) * f(c) < 0:
                b = c
            else:
                a = c
    return c

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_96924/905892170.py:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (a < b, f(a)*f(b) < 0)


In [12]:
price = bsPrice(S=100, r=0.02, vol=0.1, T=1.0, K=90, payoffType=PayoffType.Call)
f = lambda vol: (bsPrice(100, 0.02, vol, PayoffType.Call, 90, 1.0) - price)
a, b = 0.0001, 0.5
iv = bisect(f, a, b, 1e-6)
print("implied vol = ", iv)

(0.0001, 0.5)
(0.0001, 0.25005)
(0.0001, 0.125075)
(0.06258749999999999, 0.125075)
(0.09383124999999999, 0.125075)
(0.09383124999999999, 0.10945312499999998)
(0.09383124999999999, 0.10164218749999998)
(0.09773671874999998, 0.10164218749999998)
(0.09968945312499998, 0.10164218749999998)
(0.09968945312499998, 0.10066582031249999)
(0.09968945312499998, 0.10017763671874999)
(0.09993354492187498, 0.10017763671874999)
(0.09993354492187498, 0.10005559082031248)
(0.09999456787109373, 0.10005559082031248)
(0.09999456787109373, 0.10002507934570311)
(0.09999456787109373, 0.10000982360839841)
(0.09999456787109373, 0.10000219573974607)
(0.0999983818054199, 0.10000219573974607)
implied vol =  0.10000028877258299


#### 切分法

In [13]:
def secant(f, a, b, tol, maxIter):
    nIter = 0
    c = (a*f(b)-b*f(a))/(f(b)-f(a))
    while abs(a - b) > tol and nIter <= maxIter:
        print(f"({a}, {b})")
        c = (a*f(b)-b*f(a))/(f(b)-f(a))
        if abs(f(c)) < tol:
            return c
        else:
            a=b 
            b=c
        nIter = nIter+1
    return c

In [14]:
price = bsPrice(S=100, r=0.02, vol=0.1, T=1.0, K=90, payoffType=PayoffType.Call)
f = lambda vol: (bsPrice(100, 0.02, vol, PayoffType.Call, 90, 1.0) - price)
a, b = 0.0001, 0.5
iv = secant(f, a, b, 1e-6, 100)
print("implied vol = ", iv)

(0.0001, 0.5)
(0.5, 0.017869297108060234)
(0.017869297108060234, 0.035006972052249376)
(0.035006972052249376, 58.158878921372114)
(58.158878921372114, 0.3453511995504188)
(0.3453511995504188, -4.956236282273593)
(-4.956236282273593, -0.021209391295258204)
(-0.021209391295258204, 0.6740298315446954)
(0.6740298315446954, 0.2523821318154071)
(0.2523821318154071, 0.13094505724390695)
(0.13094505724390695, 0.10933139199626062)
(0.10933139199626062, 0.10146153130939682)
(0.10146153130939682, 0.10009392312534648)
(0.10009392312534648, 0.10000105609873301)
implied vol =  0.10000000077724093


#### 虚位法

In [15]:
def falsi(f, a, b, tol):
    assert (a<b and f(a)*f(b)<0)
    c = (a*f(b)-b*f(a))/(f(b)-f(a))
    while abs(a - b) > tol:
        print(f"({a}, {b})")
        c = (a*f(b)-b*f(a))/(f(b)-f(a))
        if abs(f(c)) < tol:
            return c;
        else:
            if f(a)*f(c)<0:
                b=c 
            else:
                a=c
    return c

In [16]:
price = bsPrice(S=100, r=0.02, vol=0.1, T=1.0, K=90, payoffType=PayoffType.Call)
f = lambda vol: (bsPrice(100, 0.02, vol, PayoffType.Call, 90, 1.0) - price)
a, b = 0.0001, 0.5
iv = falsi(f, a, b, 1e-6)
print("implied vol = ", iv)

(0.0001, 0.5)
(0.017869297108060234, 0.5)
(0.035006972052249376, 0.5)
(0.05153077746362442, 0.5)
(0.06708256944568117, 0.5)
(0.08007982262509539, 0.5)
(0.08913380620003186, 0.5)
(0.09448567499465944, 0.5)
(0.09731677479452577, 0.5)
(0.09872271630291884, 0.5)
(0.0993985262315561, 0.5)
(0.09971823002749905, 0.5)
(0.09986832309001992, 0.5)
(0.09993853528505835, 0.5)
(0.09997132463615187, 0.5)
(0.09998662532839857, 0.5)
(0.09999376255805052, 0.5)
(0.09999709125124431, 0.5)
(0.09999864357791168, 0.5)
(0.09999936747405776, 0.5)
(0.09999970504251011, 0.5)
(0.099999862456714, 0.5)
(0.09999993586149365, 0.5)
implied vol =  0.09999997009126486


#### 布伦特法

In [17]:
from scipy import optimize 
price = bsPrice(S=100, r=0.02, vol=0.1, T=1.0, K=90, payoffType=PayoffType.Call)
f = lambda vol: (bsPrice(100, 0.02, vol, PayoffType.Call, 90, 1.0) - price)
a, b = 0.0001, 0.5
iv = optimize.brentq(f, a, b)
print("implied vol = ", iv)

implied vol =  0.09999999999997611
